In [1]:
import pandas as pd
from datetime import datetime

In [2]:
gps = pd.read_csv("C:/dashboard_onibus/dados/dados_gps.csv", sep=";")
faixas = pd.read_csv("C:/dashboard_onibus/dados/faixas_horarias.csv", sep=";")
linhas = pd.read_csv("C:/dashboard_onibus/dados/linhas_e_nucleos.csv", sep=";")
programacao = pd.read_csv("C://dashboard_onibus/dados/programacao_partidas.csv", sep=";")

In [3]:
del gps['Linha Informada a SMTR']
del gps['Linha Realizada pela SMTR']
del gps['Veículo Consolidado pela SMTR']
del gps ['Início da Viagem pela SMTR']
del gps ['Término da Viagem pela SMTR']
del gps ['Viagem Reconhecida']

In [4]:
gps_nucleo = pd.merge(gps, linhas, on="Linha", how="left")

In [5]:
del gps_nucleo['Tipo Linha']
del gps_nucleo['Nome_Linha']

In [6]:
gps_nucleo['Início da Viagem'] = pd.to_datetime(gps_nucleo['Início da Viagem'], dayfirst=True)

In [7]:
gps_nucleo['Término da Viagem'] = pd.to_datetime(gps_nucleo['Término da Viagem'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

In [8]:
# 1. Extrair a hora da viagem como time
gps_nucleo['hora_viagem'] = gps_nucleo['Início da Viagem'].dt.time

In [9]:
# 2. Adicionar chave auxiliar para cross join
gps_nucleo['_key'] = 1
faixas['_key'] = 1

In [10]:
# Cross join
merged = pd.merge(gps_nucleo, faixas, on='_key')

In [11]:
# Converte hora_inicio e hora_fim para tipo time
merged['hora_inicio'] = pd.to_datetime(merged['hora_inicio'], format='%H:%M:%S').dt.time
merged['hora_fim'] = pd.to_datetime(merged['hora_fim'], format='%H:%M:%S').dt.time

In [12]:
# Filtra registros com hora_viagem dentro do intervalo da faixa horária
resultado = merged[(merged['hora_viagem'] >= merged['hora_inicio']) & (merged['hora_viagem'] <= merged['hora_fim'])]

In [13]:
# Selecionar colunas necessárias
resultado = resultado[['Início da Viagem', 'Faixa_Horaria']]

In [14]:
# Mescla com gps_nucleo
gps_nucleo = gps_nucleo.merge(resultado, on='Início da Viagem', how='left')

In [15]:
# Remove duplicatas com base em todas as colunas
gps_nucleo = gps_nucleo.drop_duplicates()

In [16]:
# Exporta para CSV
gps_nucleo.to_csv("C:/dashboard_onibus/gps_com_faixa_horaria.csv", sep=";", index=False, encoding='utf-8-sig')

In [17]:
# Faz a contagem de partidas por linha e faixa horária
partidas_executadas = gps_nucleo.groupby(['Linha', 'Faixa_Horaria']).size().reset_index(name='Total_Partidas')

In [18]:
# Ordena por Linha e Faixa_Horaria para melhor visualização
partidas_executadas = partidas_executadas.sort_values(by=['Linha', 'Faixa_Horaria'])

In [19]:
# (Opcional) Exporta para CSV
partidas_executadas.to_csv("C:/dashboard_onibus/partidas_executadas.csv", sep=";", index=False, encoding='utf-8-sig')

In [20]:
# 2. Garantir que as colunas estejam sem espaços
programacao.columns = programacao.columns.str.strip()
partidas_executadas.columns = partidas_executadas.columns.str.strip()

In [21]:
# 3. Renomear a coluna da contagem de executadas (se ainda não estiver)
# Supondo que partidas_executadas tenha as colunas: 'Linha', 'Faixa_Horaria', 'Partidas'
partidas_executadas = partidas_executadas.rename(columns={'Total_Partidas': 'Executado'})

In [22]:
partidas_executadas

,Linha,Faixa_Horaria,Executado
0,108,03-06,14
1,108,06-09,35
2,108,09-12,33
3,108,12-15,28
4,108,15-18,30
...,...,...,...
81,SV112,09-12,16
82,SV112,12-15,17
83,SV112,15-18,16
84,SV112,18-21,17


In [23]:


programacao = programacao.rename(columns={'Faixa': 'Faixa_Horaria', 'linha': 'Linha'})

In [24]:
programacao

,Linha,partidas_programadas,tipo_linha,tipo_dia,Faixa_Horaria
0,100,0,Linha,Dia Útil,00-03
1,105,0,Linha,Dia Útil,00-03
2,108,0,Linha,Dia Útil,00-03
3,110,0,Linha,Dia Útil,00-03
4,112,0,Linha,Dia Útil,00-03
...,...,...,...,...,...
211,SN538,5,Linha,Dia Útil,21-00
212,SP309,0,Serviço,Dia Útil,21-00
213,SP315,0,Serviço,Dia Útil,21-00
214,SP553,0,Serviço,Dia Útil,21-00


In [25]:
comparativo = pd.merge(
    partidas_executadas,
    programacao.rename(columns={'partidas_programadas': 'Programado'}),
    how='left',
    on=['Linha', 'Faixa_Horaria']
)

In [26]:
comparativo

,Linha,Faixa_Horaria,Executado,Programado,tipo_linha,tipo_dia
0,108,03-06,14,NaN,NaN,NaN
1,108,06-09,35,NaN,NaN,NaN
2,108,09-12,33,NaN,NaN,NaN
3,108,12-15,28,NaN,NaN,NaN
4,108,15-18,30,35.0,Linha,Dia Útil
...,...,...,...,...,...,...
81,SV112,09-12,16,NaN,NaN,NaN
82,SV112,12-15,17,NaN,NaN,NaN
83,SV112,15-18,16,18.0,Linha,Dia Útil
84,SV112,18-21,17,20.0,Linha,Dia Útil


In [27]:
# 5. Corrigir possíveis NaN em Programado (caso tenha faixa que não existia na programação)
comparativo['Programado'] = comparativo['Programado'].fillna(0).astype(int)

In [28]:
# Calcular o percentual de execução
comparativo['% Execução'] = (comparativo['Executado'] / comparativo['Programado'].replace(0, 1)) * 100
comparativo['% Execução'] = comparativo['% Execução'].round(1)

In [29]:
# Se programado era zero, deixar o percentual em zero
comparativo.loc[comparativo['Programado'] == 0, '% Execução'] = 0

In [30]:
del comparativo ['tipo_linha']

In [31]:
comparativo

,Linha,Faixa_Horaria,Executado,Programado,tipo_dia,% Execução
0,108,03-06,14,0,NaN,0.0
1,108,06-09,35,0,NaN,0.0
2,108,09-12,33,0,NaN,0.0
3,108,12-15,28,0,NaN,0.0
4,108,15-18,30,35,Dia Útil,85.7
...,...,...,...,...,...,...
81,SV112,09-12,16,0,NaN,0.0
82,SV112,12-15,17,0,NaN,0.0
83,SV112,15-18,16,18,Dia Útil,88.9
84,SV112,18-21,17,20,Dia Útil,85.0


In [32]:
# Exportar para CSV
comparativo.to_csv('comparativo_programado_executado.csv', index=False, sep=';')